In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys


current_working_directory = os.getcwd()
#print(f"Текущая рабочая директория Jupyter: {current_working_directory}")

# Путь к папке Project_GameDataAnalysis
# ноутбук находится в Project_GameDataAnalysis/notebooks/
# Если Jupyter запущен из Project_GameDataAnalysis:
if os.path.basename(current_working_directory) == 'Project_GameDataAnalysis':
    project_root_path = current_working_directory
elif os.path.basename(current_working_directory) == 'notebooks' and \
     os.path.basename(os.path.dirname(current_working_directory)) == 'Project_GameDataAnalysis':
    project_root_path = os.path.dirname(current_working_directory)
# Если Jupyter запущен из Data_Analytics_Portfolio:
elif os.path.basename(current_working_directory) == 'Data_Analytics_Portfolio':
    project_root_path = os.path.join(current_working_directory, 'Project_GameDataAnalysis')
else:
    # Запасной вариант 
    print("Убедитесь, что Jupyter запущен из Data_Analytics_Portfolio или Project_GameDataAnalysis.")
    project_root_path = None # Устанавливаем None, чтобы предотвратить дальнейшие ошибки

if project_root_path:
    # Добавляем корневую папку Project_GameDataAnalysis в sys.path для импорта load_data.py
    if project_root_path not in sys.path:
        sys.path.append(project_root_path)
        print(f"'{project_root_path}' добавлен в sys.path.")
    else:
        #print(f"'{project_root_path}' уже в sys.path.")
        print(f" уже в sys.path.")

    # Импортируем нашу функцию для загрузки данных из скрипта load_data.py
    # load_data.py находится непосредственно в Project_GameDataAnalysis
    from load_data import load_game_data

    # Загружаем данные, передавая project_root_path как base_path
    # Теперь load_data.py будет искать папку 'data' относительно project_root_path
    df = load_game_data(base_path=project_root_path)

    # Проверяем, что данные загрузились
    if df is not None:
        print("\nДанные успешно загружены в ноутбук!")
        print("\nПервые 5 строк данных:")
        from IPython.display import display # Убедимся, что display импортирован
        display(df.head())
        print("\nИнформация о столбцах:")
        df.info()
    else:
        print("Не удалось загрузить данные. Проверьте пути.")
else:
    print("Не удалось определить корневую директорию проекта. Данные не загружены.")
    df = None # Устанавливаем df в None, чтобы избежать ошибок

In [ ]:
# Проверим количество уникальных значений в столбцах типа 'object'
for col in df.select_dtypes(include='object').columns:
    print(f"Столбец '{col}': {df[col].nunique()} уникальных значений")
    if df[col].nunique() < 50: # Смотрим, если меньше 50
        print(df[col].value_counts(dropna=False)) # dropna=False смотрим и NaN
    print("-" * 30)

# Посмотрим на статистические сводки для числовых столбцов
print("\nСтатистические сводки для числовых столбцов:")
print(df.describe())

# --- Вставляем сюда проверку после обработки Name и Genre ---
# Количество пропущенных значений по столбцам ДО обработки Name и Genre
print("\nКоличество пропущенных значений ДО обработки Name и Genre:")
print(df.isnull().sum())

# Удаляем строки, где Name или Genre являются NaN
rows_before_name_genre_dropna = len(df)
df.dropna(subset=['Name', 'Genre'], inplace=True)
rows_after_name_genre_dropna = len(df)
print(f"\nУдалено {rows_before_name_genre_dropna - rows_after_name_genre_dropna} строк из-за NaN в 'Name' или 'Genre'.")

# Проверяем состояние DataFrame после удаления строк по Name и Genre
print("\nСостояние DataFrame после удаления строк с NaN в Name/Genre:")
print(df.info())
print("\nРазмер DataFrame после удаления по Name/Genre:", df.shape)

# --- Добавим специфическую диагностику для Year_of_Release здесь ---
# Теперь, когда Name и Genre обработаны, давайте посмотрим на Year_of_Release
print("\n*** Диагностика Year_of_Release ***")
print("Уникальные значения в 'Year_of_Release' до преобразования:")
# Увеличим head() на всякий случай, если есть много разных строковых представлений дат
print(df['Year_of_Release'].value_counts(dropna=False).head(50))
print("-" * 30)

In [ ]:
# --- Обработка User_Score ---
print("Обработка столбца 'User_Score'...")
# Заменяем 'tbd' в User_Score на NaN
initial_tbd_count = (df['User_Score'] == 'tbd').sum()
df['User_Score'] = df['User_Score'].replace('tbd', np.nan)
print(f"Заменено '{initial_tbd_count}' значений 'tbd' на NaN в 'User_Score'.")

# Преобразуем User_Score в числовой тип (float)
# errors='coerce' здесь важен, если есть другие нечисловые значения
df['User_Score'] = pd.to_numeric(df['User_Score'], errors='coerce')

# --- Обработка Year_of_Release ---
print("\nОбработка столбца 'Year_of_Release'...")

# Преобразуем столбец 'Year_of_Release' в datetime формат
# errors='coerce' заменит некорректные даты на NaT (Not a Time)
df['Year_of_Release'] = pd.to_datetime(df['Year_of_Release'], errors='coerce')

# Извлекаем год из столбца datetime
# .dt.year позволяет получить год
df['Year_of_Release'] = df['Year_of_Release'].dt.year

# Удаляем строки, где 'Year_of_Release' стало NaN после преобразования (некорректные или пустые даты)
rows_before_year_dropna = len(df)
df.dropna(subset=['Year_of_Release'], inplace=True)
rows_after_year_dropna = len(df)
print(f"Удалено {rows_before_year_dropna - rows_after_year_dropna} строк из-за NaN в 'Year_of_Release' (после обработки даты).")

# Преобразуем в целочисленный тип 
df['Year_of_Release'] = df['Year_of_Release'].astype(int)

# --- Проверка результата ---
print("\nСостояние DataFrame после обработки User_Score и Year_of_Release:")
print(df.info())
print("\nРазмер DataFrame:", df.shape)
print("\nПервые 5 строк с обновленными 'User_Score' и 'Year_of_Release':")
from IPython.display import display
display(df[['Name', 'User_Score', 'Year_of_Release']].head())